# Factorizations and other fun
Based on work by Andreas Noack

## Outline
 - Factorizations
 - Special matrix structures
 - Generic linear algebra

Before we get started, let's set up a linear system

In [1]:
A = rand(3, 3)
x = fill(1, (3,))
b = A * x

3-element Array{Float64,1}:
 0.7542467019485679
 0.9402974350372286
 1.011034219810688 

## Factorizations

#### LU factorizations
In Julia we can perform an LU factorization
```julia
PA = LU
``` 
where `P` is a permutation matrix, `L` is lower triangular unit diagonal and `U` is upper triangular, using `lu`.

Julia allows computing the LU factorization and defines a composite factorization type for storing it.

In [3]:
using LinearAlgebra

In [5]:
Alu = lu(A)

LU{Float64,Array{Float64,2}}
L factor:
3×3 Array{Float64,2}:
 1.0          0.0       0.0
 0.00998072   1.0       0.0
 0.90706     -0.204554  1.0
U factor:
3×3 Array{Float64,2}:
 0.8916  0.112208  0.00722628
 0.0     0.255253  0.488903  
 0.0     0.0       0.175449  

In [6]:
typeof(Alu)

LU{Float64,Array{Float64,2}}

The different parts of the factorization can be extracted by accessing each matrix.

In [12]:
Alu.P

3×3 Array{Float64,2}:
 0.0  0.0  1.0
 1.0  0.0  0.0
 0.0  1.0  0.0

In [11]:
Alu.L

3×3 Array{Float64,2}:
 1.0          0.0       0.0
 0.00998072   1.0       0.0
 0.90706     -0.204554  1.0

In [13]:
Alu.U

3×3 Array{Float64,2}:
 0.8916  0.112208  0.00722628
 0.0     0.255253  0.488903  
 0.0     0.0       0.175449  

Julia can dispatch methods on factorization objects.

For example, we can solve the linear system using either the original matrix or the factorization object.

In [14]:
A\b

3-element Array{Float64,1}:
 1.0
 1.0
 1.0

In [15]:
Alu\b

3-element Array{Float64,1}:
 1.0
 1.0
 1.0

Similarly, we can calculate the determinant of `A` using either `A` or the factorization object

In [16]:
det(A) ≈ det(Alu)

true

#### QR factorizations

In Julia we can perform a QR factorization
```
A=QR
``` 

where `Q` is unitary/orthogonal and `R` is upper triangular, using `qrfact`. 

In [17]:
Aqr = qr(A)

LinearAlgebra.QRCompactWY{Float64,Array{Float64,2}}
Q factor:
3×3 LinearAlgebra.QRCompactWYQ{Float64,Array{Float64,2}}:
 -0.0073924   0.988711   -0.149653
 -0.67183    -0.11576    -0.731604
 -0.740668    0.0951329   0.665102
R factor:
3×3 Array{Float64,2}:
 -1.20378  -0.118305  -0.064055
  0.0       0.258416   0.47465 
  0.0       0.0       -0.128359

The matrices `Q` and `R` can be extracted from the QR factorization object via

In [18]:
Aqr.Q

3×3 LinearAlgebra.QRCompactWYQ{Float64,Array{Float64,2}}:
 -0.0073924   0.988711   -0.149653
 -0.67183    -0.11576    -0.731604
 -0.740668    0.0951329   0.665102

In [19]:
Aqr.R

3×3 Array{Float64,2}:
 -1.20378  -0.118305  -0.064055
  0.0       0.258416   0.47465 
  0.0       0.0       -0.128359

#### Eigendecompositions

The results from eigendecompositions, singular value decompositions, Hessenberg factorizations, and Schur decompositions are all stored in `Factorization` types.

The eigendecomposition can be computed

In [22]:
Asym = A + A'
AsymEig = eigen(Asym)

Eigen{Float64,Float64,Array{Float64,2},Array{Float64,1}}
eigenvalues:
3-element Array{Float64,1}:
 -1.6237250310716922 
 -0.11993677314493456
  1.8750450275801562 
eigenvectors:
3×3 Array{Float64,2}:
 -0.7275     0.0525197  -0.684095
  0.385801  -0.793191   -0.471175
  0.567364   0.606704   -0.556784

The values and the vectors can be extracted from the Eigen type by special indexing

In [24]:
AsymEig.values

3-element Array{Float64,1}:
 -1.6237250310716922 
 -0.11993677314493456
  1.8750450275801562 

In [25]:
AsymEig.vectors

3×3 Array{Float64,2}:
 -0.7275     0.0525197  -0.684095
  0.385801  -0.793191   -0.471175
  0.567364   0.606704   -0.556784

Once again, when the factorization is stored in a type, we can dispatch on it and write specialized methods that exploit the properties of the factorization, e.g. that $A^{-1}=(V\Lambda V^{-1})^{-1}=V\Lambda^{-1}V^{-1}$.

In [26]:
inv(AsymEig)*Asym

3×3 Array{Float64,2}:
  1.0          -2.15106e-16  -6.11056e-16
  1.06581e-14   1.0          -1.38223e-14
 -8.88178e-15  -1.44329e-15   1.0        

## Special matrix structures
Matrix structure is very important in linear algebra. To see *how* important it is, let's work with a larger linear system

In [27]:
n = 1000
A = randn(n,n);

Julia can often infer special matrix structure

In [28]:
Asym = A + A'
issymmetric(Asym)

true

but sometimes floating point error might get in the way.

In [29]:
Asym_noisy = copy(Asym)
Asym_noisy[1,2] += 5eps()

1.739835830732103

In [30]:
issymmetric(Asym_noisy)

false

Luckily we can declare structure explicitly with, for example, `Diagonal`, `Triangular`, `Symmetric`, `Hermitian`, `Tridiagonal` and `SymTridiagonal`.

In [31]:
Asym_explicit = Symmetric(Asym_noisy);

Let's compare how long it takes Julia to compute the eigenvalues of `Asym`, `Asym_noisy`, and `Asym_explicit`

In [32]:
@time eigvals(Asym);

  0.135239 seconds (112.22 k allocations: 13.368 MiB)


In [33]:
@time eigvals(Asym_noisy);

  0.842813 seconds (18 allocations: 7.921 MiB)


In [34]:
@time eigvals(Asym_explicit);

  0.088056 seconds (7.62 k allocations: 8.378 MiB)


In this example, using `Symmetric()` on `Asym_noisy` made our calculations about `5x` more efficient :)

#### A big problem
Using the `Tridiagonal` and `SymTridiagonal` types to store tridiagonal matrices makes it possible to work with potentially very large tridiagonal problems. The following problem would not be possible to solve on a laptop if the matrix had to be stored as a (dense) `Matrix` type.

In [35]:
n = 1_000_000;
A = SymTridiagonal(randn(n), randn(n-1));
@time eigmax(A)

  0.757933 seconds (484.48 k allocations: 206.800 MiB, 14.77% gc time)


6.394334819676226

## Generic linear algebra
The usual way of adding support for numerical linear algebra is by wrapping BLAS and LAPACK subroutines. For matrices with elements of `Float32`, `Float64`, `Complex{Float32}` or `Complex{Float64}` this is also what Julia does.

However, Julia also supports generic linear algebra, allowing you to, for example, work with matrices and vectors of rational numbers.

#### Rational numbers
Julia has rational numbers built in. To construct a rational number, use double forward slashes:

In [36]:
1//2

1//2

#### Example: Rational linear system of equations
The following example shows how linear system of equations with rational elements can be solved without promoting to floating point element types. Overflow can easily become a problem when working with rational numbers so we use `BigInt`s.

In [37]:
Arational = Matrix{Rational{BigInt}}(rand(1:10, 3, 3))/10

3×3 Array{Rational{BigInt},2}:
 4//5  3//5   3//5 
 1//2  1//1   7//10
 2//5  1//10  3//10

In [38]:
x = fill(1, 3)
b = Arational*x

3-element Array{Rational{BigInt},1}:
  2//1
 11//5
  4//5

In [39]:
Arational\b

3-element Array{Rational{BigInt},1}:
 1//1
 1//1
 1//1

In [40]:
lu(Arational)

LU{Rational{BigInt},Array{Rational{BigInt},2}}
L factor:
3×3 Array{Rational{BigInt},2}:
 1//1   0//1   0//1
 5//8   1//1   0//1
 1//2  -8//25  1//1
U factor:
3×3 Array{Rational{BigInt},2}:
 4//5  3//5   3//5  
 0//1  5//8  13//40 
 0//1  0//1  13//125

### Exercises

#### 11.1
What are the eigenvalues of matrix A?

```
A =
[
 140   97   74  168  131
  97  106   89  131   36
  74   89  152  144   71
 168  131  144   54  142
 131   36   71  142   36
]
```

In [41]:
A =
[
 140   97   74  168  131
  97  106   89  131   36
  74   89  152  144   71
 168  131  144   54  142
 131   36   71  142   36
]

eig_A = eigen(A)

Eigen{Float64,Float64,Array{Float64,2},Array{Float64,1}}
eigenvalues:
5-element Array{Float64,1}:
 -128.49322764802145 
  -55.887784553057   
   42.752167279318854
   87.16111477514494 
  542.4677301466137  
eigenvectors:
5×5 Array{Float64,2}:
 -0.147575  -0.647178  -0.010882    0.548903   -0.507907
 -0.256795   0.173068  -0.834628   -0.239864   -0.387253
 -0.185537  -0.239762   0.422161   -0.731925   -0.440631
  0.819704   0.247506   0.0273194   0.0366447  -0.514526
 -0.453805   0.657619   0.352577    0.322668   -0.364928

#### 11.2 
Create a `Diagonal` matrix from the eigenvalues of `A`.

In [43]:
Diagonal(eig_A.vectors)

5×5 Diagonal{Float64,Array{Float64,1}}:
 -0.147575   ⋅         ⋅         ⋅           ⋅      
   ⋅        0.173068   ⋅         ⋅           ⋅      
   ⋅         ⋅        0.422161   ⋅           ⋅      
   ⋅         ⋅         ⋅        0.0366447    ⋅      
   ⋅         ⋅         ⋅         ⋅         -0.364928

#### 11.3 
Create a `LowerTriangular` matrix from `A`.

In [44]:
LowerTriangular(A)

5×5 LowerTriangular{Int64,Array{Int64,2}}:
 140    ⋅    ⋅    ⋅   ⋅
  97  106    ⋅    ⋅   ⋅
  74   89  152    ⋅   ⋅
 168  131  144   54   ⋅
 131   36   71  142  36